In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import random
import statistics
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

#DataSet Split
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
#Gradient Boosting
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_percentage_error, r2_score
#Naive Linear Regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import RidgeCV
#Trigonometric features
from sklearn.preprocessing import FunctionTransformer
#Periodic Spline Features
from sklearn.preprocessing import SplineTransformer
#Pairwise interactions with splines and polynomial features
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import FeatureUnion
#Non-Linear feature interactions with kernels
from sklearn.kernel_approximation import Nystroem

from sklearn.linear_model import LinearRegression

from numpy.fft import rfft, irfft, rfftfreq
from scipy import signal, fftpack

from statsmodels.tsa.seasonal import seasonal_decompose

data = pd.read_csv("../data/daily_sunspots_time_series_1850-01_2024-05.csv", parse_dates=['date'])
data.set_index('date', inplace = True)
data = data.drop(['date_frac', 'std', 'nobs'], axis = 1)
import time